In [ ]:
from fastai.vision import *
from fastai.metrics import error_rate

In [ ]:
import fastai
fastai.__version__

In [ ]:
path = Path('../input/snakeclassification/data/train')
df = pd.read_csv(os.path.join('../input/snakeclassification', 'train_folds_1.csv'))

In [ ]:
df.head()

In [ ]:
df.target.value_counts()

In [ ]:
src = (ImageList.from_df(path=path, df=df)
        .split_from_df(col=-1)
        .label_from_df(cols='target',)
        .add_test_folder('/kaggle/input/snakeclassification/data/test')
       )

In [ ]:
tfms = get_transforms(flip_vert=True)
tfms

In [ ]:
data = (src.transform(tfms, size=224)
        .databunch(bs=64).normalize(imagenet_stats))
data

In [ ]:
data.show_batch(rows=3, figsize=(12,9))

In [ ]:
data.classes

# Model

In [ ]:
!pip install efficientnet_pytorch

In [ ]:
# from efficientnet_pytorch import EfficientNet
# model_name = 'efficientnet-b6'
# model = EfficientNet.from_pretrained(model_name)
# model
# model.add_module('_fc',nn.Linear(2304, data.c))

In [ ]:
# learn = Learner(data, model, metrics=[accuracy,FBeta(beta=1), Precision(),Recall()]).mixup().to_fp16()
# learn.split([[learn.model._conv_stem, learn.model._bn0, learn.model._blocks[:8]],
#              [learn.model._blocks[8:],learn.model._conv_head], 
#              [learn.model._bn1,learn.model._fc]])
# learn.model_dir = '/kaggle/working/models'

# B4 NS

In [ ]:
!pip install timm

In [ ]:
import timm
model = timm.create_model('tf_efficientnet_b4_ns', pretrained=True)
model.classifier = nn.Linear(1792, 2)
model

In [ ]:
learn = Learner(data, model, metrics=[accuracy,FBeta(beta=1), Precision(),Recall()]).mixup().to_fp16()
learn.split([[learn.model.conv_stem, learn.model.bn1,  learn.model.act1, learn.model.blocks[:3]],
             [learn.model.blocks[3:], learn.model.conv_head], 
             [learn.model.bn2, learn.model.act2, learn.model.global_pool, learn.model.classifier]])
learn.model_dir = '/kaggle/working/models'

In [ ]:
learn.layer_groups

In [ ]:
learn.summary()

In [ ]:
learn.model_dir

In [ ]:
lr = 2e-3

In [ ]:
learn.fit_one_cycle(20, max_lr=slice(lr/100,lr), callbacks=[callbacks.SaveModelCallback(learn, every='improvement', monitor='f_beta', name='best_eff_b4_ns_fold1')])

In [ ]:
learn.fit_one_cycle(10, lr)

In [ ]:
learn.fit_one_cycle(5, lr)

In [ ]:
learn = None
import gc
gc.collect()

In [ ]:
learn.fit_one_cycle(10,  slice(1e-06, 1e-4))

In [ ]:
learn.save('effnet-b3-fold1')

In [ ]:
from glob import glob

In [ ]:
# Noisy Student
import timm
from glob import glob

def get_learner(kfold):
    model = timm.create_model('tf_efficientnet_b4_ns', pretrained=True)
    model.classifier = nn.Linear(1792, 2)
    learn = Learner(data, model, metrics=[accuracy,FBeta(beta=1), Precision(),Recall()]).mixup().to_fp16()
    learn.split([[learn.model.conv_stem, learn.model.bn1,  learn.model.act1, learn.model.blocks[:3]],
                 [learn.model.blocks[3:], learn.model.conv_head], 
                 [learn.model.bn2, learn.model.act2, learn.model.global_pool, learn.model.classifier]])
    learn.model_dir = '/kaggle/working/models'
    learn.load(glob(f'/kaggle/input/snakeb4ns/*b4*fold{kfold}*')[0].split('.')[0])
    return learn

def get_tta_pred(learn):
    pred = learn.TTA(ds_type=DatasetType.Test)
    return pred[0]

learn0 = get_learner(0)
# learn1 = get_learner(1)
learn2 = get_learner(2)
learn3 = get_learner(3)
learn4 = get_learner(4)

pred0 = get_tta_pred(learn0)
# pred1 = get_tta_pred(learn1)
pred2 = get_tta_pred(learn2)
pred3 = get_tta_pred(learn3)
pred4 = get_tta_pred(learn4)

In [ ]:
from efficientnet_pytorch import EfficientNet

def get_learner(kfold):
    model_name = 'efficientnet-b6'
    model = EfficientNet.from_pretrained(model_name)
    model.add_module('_fc',nn.Linear(2304, data.c))
    learn = Learner(data, model, metrics=[accuracy,FBeta(beta=1), Precision(),Recall()]).mixup().to_fp16()
    learn.split([[learn.model._conv_stem, learn.model._bn0, learn.model._blocks[:8]],
                 [learn.model._blocks[8:],learn.model._conv_head], 
                 [learn.model._bn1,learn.model._fc]])
    learn.load(glob(f'/kaggle/input/snakeb4ns/*b6*fold{kfold}*')[0].split('.')[0])
    return learn

def get_tta_pred(learn):
    pred = learn.TTA(ds_type=DatasetType.Test)
    return pred[0]

learn10 = get_learner(0)

pred10 = get_tta_pred(learn10)

In [ ]:
# Noisy Student
import timm
from glob import glob

def get_learner(kfold):
    model = timm.create_model('tf_efficientnet_b4_ns', pretrained=True)
    model.classifier = nn.Linear(1792, 2)
    learn = Learner(data, model, metrics=[accuracy,FBeta(beta=1), Precision(),Recall()]).mixup().to_fp16()
    learn.split([[learn.model.conv_stem, learn.model.bn1,  learn.model.act1, learn.model.blocks[:3]],
                 [learn.model.blocks[3:], learn.model.conv_head], 
                 [learn.model.bn2, learn.model.act2, learn.model.global_pool, learn.model.classifier]])
    learn.model_dir = '/kaggle/working/models'
    learn.load(glob(f'/kaggle/input/snakeb4ns/*b4*fold{kfold}*')[0].split('.')[0])
    return learn

def get_tta_pred(learn):
    pred = learn.TTA(ds_type=DatasetType.Test)
    return pred[0]

learn11 = get_learner(6)

pred11 = get_tta_pred(learn11)

In [ ]:
# Noisy Student
import timm
from glob import glob

def get_learner(kfold):
    model = timm.create_model('tf_efficientnet_b5_ns', pretrained=True)
    model.classifier = nn.Linear(2048, 2)
    learn = Learner(data, model, metrics=[accuracy,FBeta(beta=1), Precision(),Recall()]).mixup().to_fp16()
    learn.split([[learn.model.conv_stem, learn.model.bn1,  learn.model.act1, learn.model.blocks[:3]],
                 [learn.model.blocks[3:], learn.model.conv_head], 
                 [learn.model.bn2, learn.model.act2, learn.model.global_pool, learn.model.classifier]])
    learn.model_dir = '/kaggle/working/models'
    learn.load(glob(f'/kaggle/input/snakeb4ns/*b5*fold{kfold}*')[0].split('.')[0])
    return learn

def get_tta_pred(learn):
    pred = learn.TTA(ds_type=DatasetType.Test)
    return pred[0]

learn12 = get_learner(5)

pred12 = get_tta_pred(learn12)

In [ ]:
p = {
    'b4_fold0': pred0,
    'b4_fold2': pred2,
    'b4_fold3': pred3,
    'b4_fold4': pred4,
    'b4_fold6': pred11,
    'b6_fold0': pred10,
    'b5_fold5': pred12
}
torch.save(p, 'effnet-preds.pt')

In [ ]:
tpred = torch.load('./effnet-preds.pt')
tpred

In [ ]:
pred_final = (p0 + p2 + p3 + p4 + p11 + p10 + p12)/7.
pred_final

In [ ]:
pred_final = pred_final.argmax(dim=1).numpy()

In [ ]:
test_index = []
num = len(learn0.data.test_ds)
for i in range(num):
    test_index.append(str(learn0.data.test_ds.items[i]).split('/')[-1].split('.')[0])
test_index

In [ ]:
np_test = np.array(test_index)
with open('test_names.npy', 'wb') as f:
    np.save(f, np_test)

In [ ]:
import pandas as pd
submission = pd.DataFrame({'id':test_index,'class': pred_final})
submission = submission.sort_values(by=['id'])
submission['class'] = submission['class'].apply(lambda x: 'non_venomous' if x==0 else 'venomous')
submission.head()

In [ ]:
submission.to_csv('sumission.csv', index=False)